In [1]:
import os
import random
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import albumentations
import torch.optim as optim
from albumentations.pytorch import ToTensorV2, ToTensor

from C2C.models.resnet import *
from C2C import train
from C2C.loss import KLDLoss
from C2C.eval_model import *

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
torch.manual_seed(12)
torch.cuda.manual_seed(12)
np.random.seed(12)
random.seed(12)

torch.backends.cudnn.deterministic=True

### Read Data CSV

- CSV file must contain following columns:
    - path - location of each patch
    - wsi - Unique identifier for WSI
    - label - Label of WSI (Binary 0 or 1)
    - is_valid - If WSI part of validation cohort

In [3]:
CSV_PATH = 'data/11-3-2021 celiac_normal_train_valid_split.csv'
df = pd.read_csv(CSV_PATH)

In [4]:
df.head()

,path,wsi,is_valid,label
0,/project/GutIntelligenceLab/ys5hd/MSDS/images_...,C06-28_05,0,True
1,/project/GutIntelligenceLab/ys5hd/MSDS/images_...,C10-50_01,0,True
2,/project/GutIntelligenceLab/ys5hd/MSDS/images_...,C10-48_01,0,True
3,/project/GutIntelligenceLab/ys5hd/MSDS/images_...,C15-74_04,0,True
4,/project/GutIntelligenceLab/ys5hd/MSDS/images_...,C07-40_01,0,True


### Model

In [5]:
# Initialize Model
model_ft = WSIClassifier(2, bn_track_running_stats=True)
model_ft = model_ft.to(device)

### Loss, Optimizer, and Data Transformation

In [6]:
# Data Transforms
data_transforms = albumentations.Compose([
    ToTensor()
    ])    

# Cross Entropy Loss 
criterion_ce = nn.CrossEntropyLoss()
criterion_kld = KLDLoss()
criterion_dic = {'CE': criterion_ce, 'KLD': criterion_kld}

# Observe that all parameters are being optimized
optimizer = optim.Adam(model_ft.parameters(), lr=1e-4)

### Train Model

In [11]:
model_ft = train.train_model(model_ft, 
                             criterion_dic, 
                             optimizer, 
                             df, 
                             data_transforms=data_transforms,
                             alpha=1, 
                             beta=0.01, 
                             gamma=0.01, 
                             num_epochs=1, 
                             fpath='trained/checkpoint.pt',
                             topk=True)

### Evaluate on Test Data

In [8]:
from C2C.utils import *

ckp_path = "trained/checkpoint.pt"
model_ft, optimizer = load_ckp(ckp_path, model_ft, optimizer)

In [9]:
TEST_PATH = 'data/11-3-2021 celiac_normal_test_split.csv'
df_test = pd.read_csv(TEST_PATH)

In [10]:
pred_df = eval_test(model_ft, df_test, data_transforms)

100%|██████████| 87/87 [03:37<00:00,  2.50s/it]

Test Accuracy:  0.7701149425287356
